In [1]:
import os
import cv2
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from glob import glob
from tqdm import tqdm
from copy import deepcopy

from scipy import ndimage
from skimage.measure import label

from numpy import linalg
from scipy.sparse.linalg import eigs

from skimage.exposure import match_histograms

from utils import is_wing_facing_up, segment_contour

In [2]:
# initialize with the ones we couldn't find any lines in
seen = []
problem_wings = []

In [3]:

import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
import numpy as np
import os
from PIL import Image, ImageFilter

def imageEditor(folder, filename, blur, blurRadius): 
	# Opens a image in RGB mode
	im = Image.open('{}/{}'.format(folder, filename))

	if(blur == True):
		# blur only if checkbox checked
		im = im.filter(ImageFilter.GaussianBlur(blurRadius))
		
	# show the image
	plt.figure(figsize=(8,8))
	plt.imshow(im)
	plt.axis('off')
	plt.title(filename)
	print(filename)
	plt.show()

# creating the widgets
folderWidget = widgets.Text('../2_live_bees/3_card_mask_matches/')
fns = [x for x in sorted(os.listdir(folderWidget.value)) if x not in seen]
fileWidget = widgets.Dropdown(options = fns)
# update function to update fileslist when folder name changes
def updateFilesList(*args):
	filesList = [file for file in os.listdir(folderWidget.value) if file.endswith(('.png', '.jpg', '.jpeg','.gif'))]
	fileWidget.options = filesList
	fileWidget.value = filesList[0]

# observer folder widget and call updateFilesList whenever its value changes
folderWidget.observe(updateFilesList, 'value')

interact(
	imageEditor,
	filename = fileWidget,
	folder = folderWidget,
	blur = widgets.Checkbox(value = False),
	blurRadius = widgets.FloatSlider(min = 0, max = 5, description = "Blur Radius")
)

button_prev = widgets.Button(description="Previous!")
button_next = widgets.Button(description="Next!")
button_add = widgets.Button(description="Add to list")
button_remove = widgets.Button(description="Remove from list")

output = widgets.Output()

display(button_prev)
display(button_next)
display(button_add)
display(button_remove, output)

def on_button_prev_clicked(a):
    with output:
        cur = fileWidget.get_state()['index']
        n_fps = len(fileWidget.get_state()['_options_labels'])
        if cur > 0:
            fileWidget.set_state({'index':cur - 1})
            print(f"{cur} / {n_fps}", end='\r')

def on_button_next_clicked(b):
    global seen
    with output:
        fns = fileWidget.get_state()['_options_labels']
        cur = fileWidget.get_state()['index']
        fn = fns[cur]
        n_fps = len(fileWidget.get_state()['_options_labels'])
        fileWidget.set_state({'index':cur + 1})
        print(f"{cur} / {n_fps}                             ", end='\r')

    if fn not in seen:
        seen += [fn]
    

def on_button_add_clicked(c):
    global problem_wings
    with output:
        fns = fileWidget.get_state()['_options_labels']
        cur = fileWidget.get_state()['index']
        fn = fns[cur]
        print(f"Added {fn}", end='\r')

    if fn not in problem_wings:
        problem_wings += [fn]

def on_button_remove_clicked(c):
    global problem_wings
    with output:
        fns = fileWidget.get_state()['_options_labels']
        cur = fileWidget.get_state()['index']
        fn = fns[cur]
        print(f"Removed {fn}", end='\r')
    
    problem_wings.remove(fn)

button_prev.on_click(on_button_prev_clicked)
button_next.on_click(on_button_next_clicked)
button_add.on_click(on_button_add_clicked)
button_remove.on_click(on_button_remove_clicked)

interactive(children=(Text(value='../2_live_bees/3_card_mask_matches/', description='folder'), Dropdown(descri…

Button(description='Previous!', style=ButtonStyle())

Button(description='Next!', style=ButtonStyle())

Button(description='Add to list', style=ButtonStyle())

Button(description='Remove from list', style=ButtonStyle())

Output()

In [5]:
problem_wings = ['2024_06_06_h003bee14deformed.png',
 '2024_06_06_h03bee15.png',
 '2024_06_06_h03bee17.png',
 '2024_06_06_h05bee11.png',
 '2024_06_06_h05bee21.png',
 '2024_06_06_h05bee37.png',
 '2024_06_06_h05bee39.png',
 '2024_06_07_h02bee19.png',
 '2024_06_07_h04bee41.png',
 '2024_06_07_h04bee51.png',
 '2024_06_07_h04bee52.png',
 '2024_06_10_h02b12.png',
 '2024_06_10_h02b62.png',
 '2024_06_11_h01b05.png',
 '2024_06_11_h01b21.png',
 '2024_06_11_h01b24.png',
 '2024_06_11_h01b39.png',
 '2024_06_12_h04b24.png',
 '2024_06_12_h04b32.png',
 '2024_06_12_h04b34.png',
 '2024_06_13_h05b02.png',
 '2024_06_13_h05b21.png',
 '2024_06_13_h05b26.png',
 '2024_06_13_h05b33.png',
 '2024_06_13_h05b39.png',
 '2024_06_13_h05b78.png',
 '2024_06_14_h03b12.png',
 '2024_06_14_h03b16.png',
 '2024_06_14_h03b20.png',
 '2024_06_14_h03b23.png',
 '2024_06_14_h03b32.png',
 '2024_06_17_h04b01.png',
 '2024_06_17_h04b03.png',
 '2024_06_17_h04b10.png',
 '2024_06_18_h02b12.png',
 '2024_06_18_h02b16.png',
 '2024_06_18_h02b31.png',
 '2024_06_18_h02b42.png',
 '2024_06_18_h02b43.png',
 '2024_06_18_h02b44.png',
 '2024_06_18_h02b48.png',
 '2024_06_18_h11b03.png',
 '2024_06_18_h11b11.png',
 '2024_06_18_h11b13.png',
 '2024_06_18_h11b15.png',
 '2024_06_18_h11b21.png',
 '2024_06_19_h01b16.png',
 '2024_06_19_h01b41.png',
 '2024_06_19_h01b47.png',
 '2024_06_19_h13b03.png',
 '2024_06_19_h13b28.png',
 '2024_06_19_h13b37.png',
 '2024_06_20_h03b01.png',
 '2024_06_20_h03b11.png',
 '2024_06_20_h03b16.png',
 '2024_06_20_h03b17.png',
 '2024_06_20_h03b20.png',
 '2024_06_20_h03b25.png',
 '2024_06_20_h12b01.png',
 '2024_06_20_h12b18.png',
 '2024_06_20_h14b17.png',
 '2024_06_20_h14b22.png',
 '2024_06_20_h14b27.png',
 '2024_06_20_h14b41.png',
 '2024_06_21_h05b16.png',
 '2024_06_21_h05b20.png',
 '2024_06_21_h05b36.png',
 '2024_06_21_h05b46.png',
 '2024_06_21_h05b63.png',
 '2024_06_21_h05b82.png',
 '2024_06_21_h15b13.png',
 '2024_06_21_h15b29.png',
 '2024_06_21_h15b31.png',
 '2024_06_24_h02b01.png',
 '2024_06_24_h02b12.png',
 '2024_06_24_h02b47.png',
 '2024_06_24_h02b48.png',
 '2024_06_24_h02b51.png',
 '2024_06_24_h02b67.png',
 '2024_06_24_h12b10.png',
 '2024_06_24_h12b23.png',
 '2024_06_24_h12b36.png',
 '2024_06_24_h12b42.png',
 '2024_06_25_h03b21.png',
 '2024_06_25_h15b12.png',
 '2024_06_25_h15b16.png',
 '2024_06_25_h15b19.png',
 '2024_06_25_h15b26.png',
 '2024_06_26_h04b01.png',
 '2024_06_26_h04b13.png',
 '2024_06_26_h04b25.png',
 '2024_06_26_h04b50.png',
 '2024_06_26_h11b02.png',
 '2024_06_26_h11b18.png',
 '2024_06_27_h01b02.png',
 '2024_06_27_h01b17.png',
 '2024_06_27_h01b21.png',
 '2024_06_27_h01b36.png',
 '2024_06_27_h01b41.png',
 '2024_06_27_h14b03.png',
 '2024_06_28_h13b24.png',
 '2024_06_28_h13b25.png',
 '2024_06_28_h13b37.png',
 '2024_07_01_h11b13.png',
 '2024_07_01_h11b15.png',
 '2024_07_01_h11b16.png',
 '2024_07_01_h11b17.png',
 '2024_07_01_h11b19.png',
 '2024_07_01_h11b28.png',
 '2024_07_02_h13b15.png',
 '2024_07_02_h13b21.png',
 '2024_07_02_h13b29.png',
 '2024_07_02_h23b01.png',
 '2024_07_02_h23b04.png',
 '2024_07_02_h23b11.png',
 '2024_07_02_h23b13.png',
 '2024_07_03_h15b03.png',
 '2024_07_03_h15b05.png',
 '2024_07_03_h15b24.png',
 '2024_07_03_h25b01.png',
 '2024_07_03_h25b05.png',
 '2024_07_03_h25b13.png',
 '2024_07_03_h25b14.png',
 '2024_07_03_h25b17.png',
 '2024_07_03_h25b23.png',
 '2024_07_03_h25b26.png',
 '2024_07_03_h25b27.png',
 '2024_07_04_h14b03.png',
 '2024_07_04_h22b01.png',
 '2024_07_04_h22b10.png',
 '2024_07_04_h22b21.png',
 '2024_07_04_h24b02.png',
 '2024_07_04_h24b13.png',
 '2024_07_04_h24b22.png',
 '2024_07_04_h24b23.png',
 '2024_07_04_h24b24.png',
 '2024_07_05_h12b20.png',
 '2024_07_05_h12b21.png',
 '2024_07_05_h12b23.png',
 '2024_07_05_h12b29.png',
 '2024_07_05_h12b43.png',
 '2024_07_05_h21b03.png',
 '2024_07_05_h21b13.png',
 '2024_07_05_h21b31.png',
 '2024_07_05_h21b40.png',
 '2024_07_08_h12b07.png',
 '2024_07_08_h12b10.png',
 '2024_07_08_h12b18.png',
 '2024_07_08_h12b40.png',
 '2024_07_08_h12b41.png',
 '2024_07_08_h12b43.png',
 '2024_07_08_h21b01.png',
 '2024_07_08_h21b03.png',
 '2024_07_08_h21b19.png',
 '2024_07_08_h21b27.png',
 '2024_07_08_h21b31.png',
 '2024_07_08_h21b33.png',
 '2024_07_08_h21b37.png',
 '2024_07_09_h13b28.png',
 '2024_07_09_h13b32.png',
 '2024_07_09_h22b01.png',
 '2024_07_09_h22b04.png',
 '2024_07_09_h22b16.png',
 '2024_07_09_h22b26.png',
 '2024_07_10_h15b03.png',
 '2024_07_10_h15b19.png',
 '2024_07_10_h15b31.png',
 '2024_07_10_h24b10.png',
 '2024_07_10_h24b14.png',
 '2024_07_10_h24b15.png',
 '2024_07_11_h11b20.png',
 '2024_07_11_h23b12.png',
 '2024_07_12_h14b01.png',
 '2024_07_12_h14b02.png',
 '2024_07_12_h14b21.png',
 '2024_07_12_h14b28.png',
 '2024_07_12_h14b30.png',
 '2024_07_12_h14b32.png',
 '2024_07_12_h14b39.png',
 '2024_07_12_h14b40.png',
 '2024_07_12_h25b01.png',
 '2024_07_12_h25b13.png',
 '2024_07_12_h25b19.png',
 '2024_07_12_h25b36.png',
 '2024_07_15_h22b04.png',
 '2024_07_15_h22b11.png',
 '2024_07_15_h22b13.png',
 '2024_07_15_h22b25.png',
 '2024_07_16_h23b12.png',
 '2024_07_16_h32b01.png',
 '2024_07_16_h32b18.png',
 '2024_07_16_h32b20.png',
 '2024_07_16_h32b34.png',
 '2024_07_17_h21b11.png',
 '2024_07_17_h21b23.png',
 '2024_07_17_h31b10.png',
 '2024_07_17_h31b38.png',
 '2024_07_17_h31b41.png',
 '2024_07_17_h31b48.png',
 '2024_07_17_h31b51.png',
 '2024_07_18_h24b18.png',
 '2024_07_18_h24b24.png',
 '2024_07_18_h33b01.png',
 '2024_07_18_h33b07.png',
 '2024_07_18_h33b10.png',
 '2024_07_18_h33b14.png',
 '2024_07_18_h33b21.png',
 '2024_07_18_h33b25.png',
 '2024_07_18_h33b26.png',
 '2024_07_18_h33b35.png',
 '2024_07_18_h33b39.png',
 '2024_07_18_h33b57.png',
 '2024_07_19_h25b32.png',
 '2024_07_19_h34b20.png',
 '2024_07_19_h34b28.png',
 '2024_07_19_h34b46.png',
 '2024_07_19_h35b02.png',
 '2024_07_19_h35b03.png',
 '2024_07_19_h35b12.png',
 '2024_07_19_h35b29.png',
 '2024_07_19_h35b57.png',
 '2024_07_22_h23b05.png',
 '2024_07_22_h33b01.png',
 '2024_07_22_h33b04.png',
 '2024_07_22_h33b14.png',
 '2024_07_22_h33b22.png',
 '2024_07_22_h33b24X.png',
 '2024_07_22_h33b35.png',
 '2024_07_22_h33b40.png',
 '2024_07_22_h33b55.png',
 '2024_07_23_h21b27.png',
 '2024_07_23_h21b28.png',
 '2024_07_24_h34b12.png',
 '2024_07_24_h34b13.png',
 '2024_07_24_h34b15.png',
 '2024_07_24_h34b33.png',
 '2024_07_24_h34b35.png',
 '2024_07_24_h34b37.png',
 '2024_07_24_h34b39.png',
 '2024_07_24_h34b42.png',
 '2024_07_24_h34b51.png',
 '2024_07_25_h24b02.png',
 '2024_07_25_h31b10.png',
 '2024_07_25_h31b38.png',
 '2024_07_25_h31b60.png',
 '2024_07_26_h22b13.png',
 '2024_07_26_h22b32.png',
 '2024_07_26_h32b05.png',
 '2024_07_26_h32b10.png',
 '2024_07_26_h32b19.png',
 '2024_07_26_h32b20.png',
 '2024_07_26_h32b21.png',
 '2024_07_26_h32b30.png',
 '2024_07_26_h32b37.png',
 '2024_07_29_h32b01.png',
 '2024_07_29_h32b10.png',
 '2024_07_29_h32b18.png',
 '2024_07_29_h32b26.png',
 '2024_07_29_h32b34.png',
 '2024_07_30_h35b27.png',
 '2024_07_30_h35b55.png',
 '2024_07_31_h31b19.png',
 '2024_07_31_h31b26.png',
 '2024_07_31_h31b49.png',
 '2024_07_31_h31b63.png',
 '2024_07_31_h31b67.png',
 '2024_07_31_h31b73.png',
 '2024_08_01_h34b15.png',
 '2024_08_01_h34b30.png',
 '2024_08_02_h33b07.png',
 '2024_08_02_h33b24X.png',
 '2024_08_02_h33b48.png',
 '2024_08_02_h33b59.png',
 '2024_08_05_h32b01.png',
 '2024_08_05_h32b05.png',
 '2024_08_05_h32b31.png',
 '2024_08_07_h34b12.png',
 '2024_08_07_h34b15.png',
 '2024_08_09_h31b01.png']

['2024_06_06_h003bee14deformed.png',
 '2024_06_06_h03bee15.png',
 '2024_06_06_h03bee17.png',
 '2024_06_06_h05bee11.png',
 '2024_06_06_h05bee21.png',
 '2024_06_06_h05bee37.png',
 '2024_06_06_h05bee39.png',
 '2024_06_07_h02bee19.png',
 '2024_06_07_h04bee41.png',
 '2024_06_07_h04bee51.png',
 '2024_06_07_h04bee52.png',
 '2024_06_10_h02b12.png',
 '2024_06_10_h02b62.png',
 '2024_06_11_h01b05.png',
 '2024_06_11_h01b21.png',
 '2024_06_11_h01b24.png',
 '2024_06_11_h01b39.png',
 '2024_06_12_h04b24.png',
 '2024_06_12_h04b32.png',
 '2024_06_12_h04b34.png',
 '2024_06_13_h05b02.png',
 '2024_06_13_h05b21.png',
 '2024_06_13_h05b26.png',
 '2024_06_13_h05b33.png',
 '2024_06_13_h05b39.png',
 '2024_06_13_h05b78.png',
 '2024_06_14_h03b12.png',
 '2024_06_14_h03b16.png',
 '2024_06_14_h03b20.png',
 '2024_06_14_h03b23.png',
 '2024_06_14_h03b32.png',
 '2024_06_17_h04b01.png',
 '2024_06_17_h04b03.png',
 '2024_06_17_h04b10.png',
 '2024_06_18_h02b12.png',
 '2024_06_18_h02b16.png',
 '2024_06_18_h02b31.png',
 '2024_